<a href="https://colab.research.google.com/github/gummadhav/Let_us_Learn/blob/main/CUDA_Streams_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell
%%shell
# Update package lists
apt-get update

# Install essential build tools (if not already present, good practice)
apt-get install -y build-essential

# Install OpenCV development libraries (C++ headers and shared libraries)
# 'libopencv-dev' provides the development files
# 'python3-opencv' is for Python, but sometimes pulled in as a dependency
apt-get install -y libopencv-dev python3-opencv

# Verify OpenCV version (optional, but good for checking)
pkg-config --modversion opencv4 # For OpenCV 4.x
# Or for older versions: pkg-config --modversion opencv

In [ ]:
!pip install nvcc4jupyter

In [ ]:
%load_ext nvcc4jupyter

In [ ]:
%%shell
nvcc --version

In [ ]:
%%shell
nvidia-smi

In [ ]:
%%writefile gpu_with_threaded_CUDA_Streams.cu

#include <opencv2/opencv.hpp>
#include <vector>
#include <string>
#include <filesystem>
#include <iostream>
#include <queue>
#include <mutex>
#include <thread>
#include <atomic>
#include "cuda_runtime.h"

#define MAX_STREAMS 4

// Utility: get image paths from folder
std::vector<std::string> get_image_paths(const std::string& folder) {
    std::vector<std::string> paths;
    for (const auto& entry : std::filesystem::directory_iterator(folder)) {
        if (entry.path().extension() == ".bmp") {
            paths.push_back(entry.path().string());
        }
    }
    return paths;
}

// CUDA Gaussian Filter kernel
__global__
void gaussian_filter_kernel(const uchar* input, uchar* output, int width, int height, const float* kernel) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    if (x >= width || y >= height) return;
    float sum = 0, kSum = 0;
    for (int dy = -1; dy <= 1; ++dy) {
        for (int dx = -1; dx <= 1; ++dx) {
            int ix = min(max(x+dx,0), width-1);
            int iy = min(max(y+dy,0), height-1);
            float k = kernel[(dy+1)*3 + (dx+1)];
            sum += input[iy*width + ix] * k;
            kSum += k;
        }
    }
    float result = kSum > 0 ? sum / kSum : 0;
    result = min(max(result, 0.0f), 255.0f);
    output[y*width + x] = static_cast<uchar>(result);
}

__global__
void laplacian_filter_kernel(const uchar* input, uchar* output, int width, int height) {
    int kernel[3][3] = { {-1,-1,-1}, {-1,8,-1}, {-1,-1,-1} };
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    if (x >= width || y >= height) return;
    int sum = 0;
    for (int dy = -1; dy <= 1; ++dy) {
        for (int dx = -1; dx <= 1; ++dx) {
            int ix = min(max(x+dx,0), width-1);
            int iy = min(max(y+dy,0), height-1);
            sum += input[iy*width + ix] * kernel[dy+1][dx+1];
        }
    }
    int result = sum + 128;
    result = min(max(result, 0), 255);
    output[y*width + x] = static_cast<uchar>(result);
}

#define CHECK_CUDA(call) \
    { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            std::cerr << "CUDA error: " << cudaGetErrorString(err) << " at " << __FILE__ << ":" << __LINE__ << std::endl; \
            exit(1); \
        } \
    }

// Empty kernel for warm-up
__global__ void empty_kernel(int* p) { *p = 1; }

// Warm-up function
void warm_up() {
    std::cout << "Warming up CUDA..." << std::endl;
    int* d_temp;
    CHECK_CUDA(cudaMalloc(&d_temp, sizeof(int)));
    empty_kernel<<<1, 1>>>(d_temp);  // Launch any small dummy kernel
    CHECK_CUDA(cudaDeviceSynchronize());
    CHECK_CUDA(cudaFree(d_temp));
    std::cout << "Warm-up complete." << std::endl;
}

// Buffer struct per image
struct DeviceBuffers {
    uchar *d_input = nullptr, *d_gauss = nullptr, *d_laplace = nullptr;
    float *d_gkernel = nullptr;
    cudaStream_t stream = nullptr;
    cudaEvent_t event_start = nullptr, event_stop = nullptr;
    int width = 0, height = 0;
    size_t size = 0;
};

std::queue<std::string> image_queue;
std::mutex queue_mutex;
std::atomic<int> images_processed(0);

// Per-image GPU worker
void process_image_gpu(const std::string& path, const float* gaussKernel, const std::string& output_folder) {
    cv::Mat img = cv::imread(path, cv::IMREAD_GRAYSCALE);
    if (img.empty() || img.type() != CV_8U || img.channels() != 1) {
        std::cerr << "Invalid image: " << path << std::endl;
        return;
    }
    int width = img.cols, height = img.rows;
    size_t size = width * height * sizeof(uchar);

    // Allocate device buffers and stream/events
    DeviceBuffers db;
    db.width = width;
    db.height = height;
    db.size = size;

    CHECK_CUDA(cudaMalloc(&db.d_input, size));
    CHECK_CUDA(cudaMalloc(&db.d_gauss, size));
    CHECK_CUDA(cudaMalloc(&db.d_laplace, size));
    CHECK_CUDA(cudaMalloc(&db.d_gkernel, 9 * sizeof(float)));

    CHECK_CUDA(cudaStreamCreate(&db.stream));
    CHECK_CUDA(cudaEventCreate(&db.event_start));
    CHECK_CUDA(cudaEventCreate(&db.event_stop));

    // Copy data to device asynchronously
    CHECK_CUDA(cudaMemcpyAsync(db.d_input, img.data, size, cudaMemcpyHostToDevice, db.stream));
    CHECK_CUDA(cudaMemcpyAsync(db.d_gkernel, gaussKernel, 9 * sizeof(float), cudaMemcpyHostToDevice, db.stream));

    dim3 block(16,16), grid((width+15)/16, (height+15)/16);

    // Timing start
    CHECK_CUDA(cudaEventRecord(db.event_start, db.stream));

    gaussian_filter_kernel<<<grid, block, 0, db.stream>>>(db.d_input, db.d_gauss, width, height, db.d_gkernel);
    laplacian_filter_kernel<<<grid, block, 0, db.stream>>>(db.d_gauss, db.d_laplace, width, height);

    // Timing stop
    CHECK_CUDA(cudaEventRecord(db.event_stop, db.stream));
    CHECK_CUDA(cudaGetLastError()); // Check for kernel errors after recording stop event

    // Copy output to host asynchronously
    cv::Mat out(height, width, CV_8U);
    CHECK_CUDA(cudaMemcpyAsync(out.data, db.d_laplace, size, cudaMemcpyDeviceToHost, db.stream));

    // Wait for all GPU operations
    CHECK_CUDA(cudaStreamSynchronize(db.stream));
    CHECK_CUDA(cudaGetLastError());

    // Timing calculation
    float kernel_ms = 0;
    CHECK_CUDA(cudaEventElapsedTime(&kernel_ms, db.event_start, db.event_stop));

    // Save the image
    std::string gpu_output = output_folder + "/" + std::filesystem::path(path).stem().string() + "_gpu.bmp";
    if (!cv::imwrite(gpu_output, out)) {
        std::cerr << "Failed to save GPU image: " << gpu_output << std::endl;
    }

    std::cout << "Processed [" << path << "] with GPU kernel time: " << kernel_ms << " ms" << std::endl;

    // Clean up
    CHECK_CUDA(cudaFree(db.d_input));
    CHECK_CUDA(cudaFree(db.d_gauss));
    CHECK_CUDA(cudaFree(db.d_laplace));
    CHECK_CUDA(cudaFree(db.d_gkernel));
    CHECK_CUDA(cudaEventDestroy(db.event_start));
    CHECK_CUDA(cudaEventDestroy(db.event_stop));
    CHECK_CUDA(cudaStreamDestroy(db.stream));
}

// Thread worker pool function
void worker_func(const float* gaussKernel, const std::string& output_folder) {
    while (true) {
        std::string path;
        {
            std::lock_guard<std::mutex> lock(queue_mutex);
            if (image_queue.empty())
                break;
            path = image_queue.front();
            image_queue.pop();
        }
        process_image_gpu(path, gaussKernel, output_folder);
        images_processed++;
    }
}

int main() {
    std::string input_folder = "input_images";
    std::string output_folder = "output_images";
    std::filesystem::create_directories(output_folder);

    auto paths = get_image_paths(input_folder);
    if (paths.empty()) {
        std::cerr << "No BMP images found in " << input_folder << std::endl;
        return 1;
    }

    for (const auto& p : paths)
        image_queue.push(p);

    float gaussKernel[9] = {1.f/16, 2.f/16, 1.f/16, 2.f/16, 4.f/16, 2.f/16, 1.f/16, 2.f/16, 1.f/16};

    // ---- WARM-UP CALL ----
    warm_up();

    // Launch worker threads, limited by MAX_STREAMS
    int thread_count = std::min(MAX_STREAMS, (int)paths.size());
    std::vector<std::thread> workers;
    for (int i = 0; i < thread_count; ++i) {
        workers.emplace_back(worker_func, gaussKernel, output_folder);
    }
    for (auto& t : workers) t.join();

    std::cout << "All images processed. Total: " << images_processed.load() << std::endl;
    return 0;
}